In [2]:
import os
import json
from pathlib import Path
import random 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from transformers import (
    AutoImageProcessor,
    TFBertTokenizer,
    TFVisionTextDualEncoderModel
    )

2023-09-18 12:19:15.444129: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 12:19:15.488461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 12:19:15.489048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 12:19:16.379822: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/enrico/univr/Tesi/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_args = {
    'image_path_to_caption_file': '/home/enrico/univr/Tesi/trainer/image_path_to_caption_train.json',
    'images_dir': '/home/enrico/univr/Tesi/images',
    'vision_model_name_or_path': "google/vit-base-patch16-224",
    'text_model_name_or_path': 'bert-base-uncased',
    'train_split': .8,
    'num_train_epochs': 1,
}

In [4]:
processor = AutoImageProcessor.from_pretrained(model_args['vision_model_name_or_path'])
tokenizer = TFBertTokenizer.from_pretrained(model_args['text_model_name_or_path'])

model = TFVisionTextDualEncoderModel.from_vision_text_pretrained(text_model_name_or_path=model_args['text_model_name_or_path']
        ,vision_model_name_or_path= model_args['vision_model_name_or_path'] 
    )

# if model_args.freeze_vision_model:
model.vision_model.trainable = False

# if model_args.freeze_text_model:
model.text_model.trainable = False


2023-09-18 12:19:20.447997: I tensorflow_text/core/kernels/fast_bert_normalizer_model_builder.cc:211] CharacterSet built (lower_case_nfd_strip_accents=1). Trie data size (int32): 35072. Normalized string pool size (byte): 111208
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should p

In [4]:
# group all captions together having the same image path
def load_img_text_pairs(image_path_to_caption_file,images_dir):
    with open(image_path_to_caption_file, 'r') as f:
        image_path_to_caption_file = json.load(f)
    captions = []
    images = []
    for image_path in image_path_to_caption_file.keys():
        caption_list = image_path_to_caption_file[image_path]
        captions.extend(caption_list)
        images.extend([os.path.join(images_dir,image_path)] * len(caption_list))  
    return (images, captions)

In [5]:
def standardize_text(inputs):
  return tf.strings.regex_replace(inputs,
                                  r"!\"#$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~", "")

In [6]:
def crop_to_square(image):
    height, width = tf.shape(image)[0], tf.shape(image)[1]
    if height > width:
        image = tf.image.crop_to_bounding_box(
            image, (height - width) // 2, 0, width, width)
    elif width > height:
        image = tf.image.crop_to_bounding_box(
            image, 0, (width - height) // 2, height, height)
    return image

In [17]:
x = tf.constant([[[1, 2, 3,4], [ 5, 6,7,8]],[[1, 2, 3,4], [ 5, 6,7,8]],[[1, 2, 3,4], [ 5, 6,7,8]]])
x = tf.transpose(x, perm=[2,0,1])
x = tf.transpose(x, perm=[1,2,0])
x

<tf.Tensor: shape=(3, 2, 4), dtype=int32, numpy=
array([[[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]]], dtype=int32)>

In [20]:
def create_tf_datasets(image_path_to_caption_file, images_dir, image_processor, caption_tokenizer, train_batch_size=64, test_batch_size=32, train_split=.8):
    images, captions = load_img_text_pairs(image_path_to_caption_file, images_dir)
    ds = standardize_text(captions)
    ds = caption_tokenizer(captions)
    del ds['token_type_ids']
    ds['image_path'] = images
    tf_dataset = tf.data.Dataset.from_tensor_slices(ds)
    
    def load_image(sample):
        image_path = sample['image_path']
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(
            image, channels=3, expand_animations=False)
        image = crop_to_square(image)
        image = tf.image.resize(
            image, list(image_processor.size.values()), method="bicubic", antialias=True)
        image = image / 255.0
        image = (image - image_processor.image_mean) / image_processor.image_std
        # Convert to channels-first
        image = tf.transpose(image, perm=[2, 0, 1])
        sample["pixel_values"] = image
        del sample['image_path']
        return sample

    def transform(sample, seed):
        image = load_image(sample)['pixel_values']
        new_seed = tf.random.experimental.stateless_split(seed, num=1)[0]
        # Random brightness
        image = tf.image.stateless_random_brightness(
            image, max_delta=0.5, seed=new_seed)
        # Random contrast
        image = tf.image.stateless_random_contrast(
            image, lower=0.1, upper=0.9, seed=new_seed)
        image = tf.transpose(image, perm=[1, 2, 0])
        image = tf.image.stateless_random_hue(image,.25,seed=seed)
        # Rotation
        image = tf.image.rot90(image)
        image = tf.transpose(image, perm=[2, 0, 1])
        sample["pixel_values"] = image
        return sample

    # Create a generator.
    rng = tf.random.Generator.from_seed(123, alg='philox')

    # Create a wrapper function for updating seeds.
    def update_seed(x):
        seed = rng.make_seeds(2)[0]
        image= transform(x, seed)
        print(image)
        return image

    train_len = round(len(tf_dataset) * float(train_split))
    shuffled = tf_dataset.shuffle(len(tf_dataset), seed=42, reshuffle_each_iteration=False)
    train = shuffled\
                    .take(train_len)\
                    .map(update_seed, num_parallel_calls=tf.data.AUTOTUNE)\
                    .batch(train_batch_size)\
                    .prefetch(tf.data.AUTOTUNE)
    test = shuffled\
                    .skip(train_len)\
                    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)\
                    .batch(test_batch_size)\
                    .prefetch(tf.data.AUTOTUNE)

    return train, test

In [21]:
train_dataset, val_dataset = create_tf_datasets(model_args['image_path_to_caption_file'], 
                                                model_args['images_dir'], 
                                                processor, 
                                                tokenizer, 
                                                train_split=model_args['train_split'])
next(iter(train_dataset))

{'input_ids': <tf.Tensor 'args_2:0' shape=(189,) dtype=int64>, 'attention_mask': <tf.Tensor 'args_0:0' shape=(189,) dtype=int64>, 'pixel_values': <tf.Tensor 'transpose_2:0' shape=(3, 224, 224) dtype=float32>}


{'input_ids': <tf.Tensor: shape=(64, 189), dtype=int64, numpy=
 array([[  101,  3393, 21877, ...,     0,     0,     0],
        [  101, 22949,  5428, ...,     0,     0,     0],
        [  101,  2009,  2036, ...,     0,     0,     0],
        ...,
        [  101,  2009,  2036, ...,     0,     0,     0],
        [  101,  9665,  2317, ...,     0,     0,     0],
        [  101, 10250, 19629, ...,     0,     0,     0]])>,
 'attention_mask': <tf.Tensor: shape=(64, 189), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 'pixel_values': <tf.Tensor: shape=(64, 3, 224, 224), dtype=float32, numpy=
 array([[[[ 0.08401372,  0.092843  ,  0.03194861, ...,  0.51255864,
            0.5138095 ,  0.5283047 ],
          [ 0.10082122,  0.13096133,  0.14288099, ...,  0.5245258 ,
            0.51931965,  0.5209845 ],
         

In [22]:
model.compile(optimizer='adam')

model.fit(train_dataset,
    validation_data=val_dataset,
    epochs=int(model_args['num_train_epochs'])
)

  1/193 [..............................] - ETA: 2:40:06 - loss: 4.2151